In [ ]:
# This is a test - To improve upon model accuracy.  @author Shaela , August 2022
#
#  Same dataset for bird identification using bird songs.
#
#


In [4]:
import os, sys, numpy,scipy
import sklearn
import tensorflow as tf
import tensorflow_estimator, tensorboard

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

import tensorflow.keras.utils
import tensorflow.keras.activations
import tensorflow.keras.applications
import tensorflow.keras.callbacks
import tensorflow.keras.initializers
from tensorflow.keras import losses,metrics
from tensorflow.keras.applications import ResNet50

# Audio file processing
import librosa
import librosa.display
import soundfile as sf


# EXTRA EXTRA - because they don't work upstairs?
import pydub
from pydub import AudioSegment
import IPython
from IPython.display import Audio
#warnings.filterwarnings('ignore')


# Start Code Here.
print("  Sweet are the beginnings !! ")
print("********** !**********")


  Sweet are the beginnings !! 
********** !**********


In [15]:

# The Convolutional Base of the Pre-Trained Model will be added as a Layer in this Model
Conv_Base = tf.keras.applications.resnet50.ResNet50(include_top = False, weights = 'imagenet', input_shape = (150,150, 3))

for layer in Conv_Base.layers[:-8]:
    layer.trainable = False

model = Sequential()
model.add(Conv_Base)
model.add(Flatten())
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation = 'softmax'))

model.summary()
print("------Model Summary----------------------------------")

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_5 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               13107456  
_________________________________________________________________
dense_13 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 20)                5140      
Total params: 36,766,100
Trainable params: 16,593,940
Non-trainable params: 20,172,160
_________________________________

In [11]:

#base_dir = 'Deep_Learning_With_Python_Book/Dogs_Vs_Cats_Small'
#if os.path.exists(base_dir):
#    train_dir = os.path.join(base_dir, 'train')
#    validation_dir = os.path.join(base_dir, 'validation')
#    test_dir = os.path.join(base_dir, 'test')
#else:
#    print("The Folder, {}, doesn't exist'".format(base_dir))


print("Create DATA generators for the model.\n")


datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2,dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input) # Training and Validation split 80/20

batch_size = 20

#Train_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)
#Val_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

train_gen = datagen.flow_from_directory(directory = './img_data2',
                                        target_size = (150,150),
                                        batch_size = batch_size,
                                        class_mode = 'categorical',subset="training",
                                        seed=42)

val_gen = datagen.flow_from_directory(directory = './img_data2',
                                      target_size = (150,150),
                                      batch_size = batch_size, class_mode = 'categorical',
                                      subset="validation",seed=42)





Create DATA generators for the model.

Found 484 images belonging to 20 classes.
Found 112 images belonging to 20 classes.


In [ ]:

# Model
rcallback = [ModelCheckpoint(filepath='./models/bestTryModel.hdf5', monitor='val_loss', save_best_only=True, mode='auto')]

epochs = 1000
Number_Of_Training_Images = train_gen.classes.shape[0]
steps_per_epoch = Number_Of_Training_Images/batch_size

model.compile(optimizer=keras.optimizers.Adam(lr=3e-4),
              loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model.fit(train_gen, epochs = epochs,
                    #batch_size = batch_size,
                    validation_data = val_gen, callbacks=rcallback,
                    steps_per_epoch = steps_per_epoch)


In [ ]:

import matplotlib.pyplot as plt

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
No_Of_Epochs = range(epochs)



plt.plot(No_Of_Epochs, train_acc, marker = 'o', color = 'blue', markersize = 12,
                 linewidth = 2, label = 'Training Accuracy')
plt.plot(No_Of_Epochs, val_acc, marker = '.', color = 'red', markersize = 12,
                 linewidth = 2, label = 'Validation Accuracy')

plt.title('Training Accuracy and Testing Accuracy w.r.t Number of Epochs')

plt.legend()

plt.figure()

plt.plot(No_Of_Epochs, train_loss, marker = 'o', color = 'blue', markersize = 12,
                 linewidth = 2, label = 'Training Loss')
plt.plot(No_Of_Epochs, val_acc, marker = '.', color = 'red', markersize = 12,
                 linewidth = 2, label = 'Validation Loss')

plt.title('Training Loss and Testing Loss w.r.t Number of Epochs')

plt.legend()

plt.show()